<a href="https://colab.research.google.com/github/RAMJIJAISWAL97/COVID-19-EDA-and-Predictive-Analysis/blob/master/covid19Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget http://cb.lk/covid_19

--2020-08-31 02:14:39--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2020-08-31 02:14:39--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2020-08-31 02:14:40--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2020-08-31 02:14:40--  https://www.dropbox.com/s/raw/7rjw

In [3]:
!unzip /content/covid_19

Archive:  /content/covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-

In [42]:
TRAIN_PATH = 'CovidDataset/Train'
VAL_PATH = 'CovidDataset/Val'

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [44]:
# CNN Based Models in Keras
model = keras.Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = (224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,  activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1,  activation = 'sigmoid'))

model.compile(loss = keras.losses.binary_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 54, 54, 64)       

In [45]:
# Training From Scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(
    rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary',
)



Found 224 images belonging to 2 classes.


In [46]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [47]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary',
)

Found 60 images belonging to 2 classes.


In [48]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 6,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 2
)

Epoch 1/10
6/6 [==============================] - 9s 1s/step - loss: 1.4562 - accuracy: 0.5625 - val_loss: 0.6600 - val_accuracy: 0.8000
Epoch 2/10
6/6 [==============================] - 8s 1s/step - loss: 0.5719 - accuracy: 0.7083 - val_loss: 0.5764 - val_accuracy: 0.7000
Epoch 3/10
6/6 [==============================] - 8s 1s/step - loss: 0.4899 - accuracy: 0.7708 - val_loss: 0.4608 - val_accuracy: 0.8833
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 0.3901 - accuracy: 0.8021 - val_loss: 0.3139 - val_accuracy: 0.9167
Epoch 5/10
6/6 [==============================] - 8s 1s/step - loss: 0.3618 - accuracy: 0.8906 - val_loss: 0.3077 - val_accuracy: 0.9500
Epoch 6/10
6/6 [==============================] - 8s 1s/step - loss: 0.2670 - accuracy: 0.8958 - val_loss: 0.1851 - val_accuracy: 0.9500
Epoch 7/10
6/6 [==============================] - 8s 1s/step - loss: 0.2992 - accuracy: 0.8854 - val_loss: 0.3521 - val_accuracy: 0.9667
Epoch 8/10
6/6 [=========================

In [49]:
model.save("model_adv.h5")

In [50]:
model.evaluate_generator(validation_generator)

[0.12543997168540955, 0.9666666388511658]

In [51]:
model.evaluate_generator(validation_generator)

[0.12543997168540955, 0.9666666388511658]

In [52]:
#Test Image

In [53]:
import os
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [63]:
y_actual = []
y_test = []

In [64]:
for i in os.listdir('./CovidDataset/Val/Covid/') :
  img = image.load_img('./CovidDataset/Val/Covid/'+i, target_size=(224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)

In [56]:
for i in os.listdir('./CovidDataset/Val/Covid/') :
  img = image.load_img('./CovidDataset/Val/Covid/'+i, target_size=(224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)

In [65]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

In [66]:
import seaborn as sns

In [67]:
from sklearn.metrics import confusion_matrix

In [68]:
cm = confusion_matrix(y_actual, y_test)

[[30]]
